In [4]:
### Quiz program test bench
### Started 2016-05-16
###            Quiz program is implemented like having 
###            deck of cards with questions and answers,
###            where the cards belong to at least one category.
###            Questions may be multiple-choice.

# =======================================
# ==  Ideas:
# ==    - main program should start 
# ==      QuizMaster ("handler" or "assistant") [represents the quiz game leader]
# ==      This is the object from which everything is controlled.
# ==    
# ==   *******
# ==    - Objects with attributes:
# ==        * QuizMaster
# ==          .. function1: run() -- start it, should load the library of questions
# ==          .. function2: quizMe() -- take a quiz 
# ==          IGNORE.. function3: quiz .. Quick quiz (ask one question, choice of category)
# ==          .. function3: addQuestionCard() -- add question card to library
# ==          .. function4: listQuestions(Category = None) -- list all Questions (of certain category or all)
# ==          .. function5: quit() -- exit QuizMaster
# == 
# ==          .. attribute1: allQuestions<Deck> -- full library of question cards [like full deck of Question cards]
# ==          .. attribute2: currentDeck<Deck>  -- deck of cards now in play [the deck from which Question cards are drawn, may be full Library or subset]
# ==          
# ==
# ==        * Deck
# ==          .. function1: load()                    -- reads all questions from text files
# ==          
# ==          .. attribute1: lib<dictionary> -- dictionary with structure {id(int): Question} 
# == 
# ==        * QuestionCard [represents card with question and answer]
# ==          .. Attribute1: thequestion<string> -- The question in text
# ==          .. Attribute2: theanswer<string> -- (in text or alternative if multiple choice)
# ==          .. Attribute3: categories<[string1,string2...]> -- list of Categories
# ==
# ==
# ==
# ==
# ==
# ==
# =======================================




# First, classes. Scroll down to see main program
### (not sure how to make other cell's content visible)

g_DEBUG = True # will print out a bunch of messages
g_PATHQUESTIONFILE = "/Users/edvinsidebo/code-projects/quizMe/Questions/allquestions.txt" # path to question text file (for now assume only one)

#import sys
import random

# Question
class QuestionCard:

    def __init__(self, question, answers, categories):
        self.quest = question
        self.answ = answers
        self.categ = categories
        # list index identifying the position of the correct alternative
        # will be set by the printAnswersInRandomisedOrder() function 
        # (only relevant for multiple-choice questions)
        self.idx_random_correct = -99
        self.letters = 'ABCDEFGHIJKLMNOP'

    def printAnswersInRandomisedOrder(self):
        indices = list(range(len(self.answ)))
        indices_rand = indices
        random.shuffle(indices_rand)
        print("Answer alternatives: ")
        idx_count = 0
        idx_correct = -99
        for idx_rand in indices_rand:
            print("{}: {}".format(self.letters[idx_count], self.answ[idx_rand]))
            if idx_rand == 0:
                idx_correct = idx_count
            idx_count += 1
            
        return idx_correct
        
    def ask(self, retry = False):
        print ("")
        # Ask question
        if not retry:
            print(self.quest)
            print("----------------------------------")
        
        # Give answer (non-multiple-choice) or ask for answer alternatives (multiple-choice)
        if not retry:
            # If more than one answer, interpret as multiple-choice question, and print answer alternatives in randomised order
            if len(self.answ) > 1:
                self.idx_random_correct = self.printAnswersInRandomisedOrder()
            # else, prompt for answer
            else:
                prompt_ans = input("Hit any key to see the answer: ")
                print("Answer: ")
                for ans_line in self.answ[0].split("\n"):
                    print("         {}".format(ans_line))
        
        # If multiple-choice, prompt for user answer
        if len(self.answ) > 1:
            try:
                user_answer = str(input("*** Please enter your answer: ")).upper()
                assert user_answer in self.letters[:len(self.answ)]
            except AssertionError:
                print("ERROR: Your answer is not among the accepted ones. Please try again: ")
                return self.ask(True)
            if user_answer == self.letters[self.idx_random_correct]:
                print("\nYou gave answer {}, which is correct! Good job!".format(user_answer))
            else:
                print("\nYou gave answer {0}, which is not correct... Sorry. The correct answer is {1}".format(user_answer, self.letters[self.idx_random_correct]))
        

        cont = str(input("Hit 'Q' to quit. Hit any other key to continue: ")).upper()
        if cont == "Q":
            return False
                   
        return True
    
    def show(self):
        print("Question: {}".format(self.quest))
        if len(self.answ) > 1:
            print("Answers: ")
            for idx,ans in enumerate(self.answ):
                print("{}: {}".format(self.letters[idx], ans))
        else: 
            print("Answer: {}".format(self.answ[0]))
        print("Categories (tags): {}".format(','.join(self.categ).lstrip(',')))

# Class for reading questions from textfile, making QuestionCards
# and putting them into a Deck
class QuestionMaker:

    path_questionfile = ""

    def __init__(self, path_questionfile):
        self.path_questionfile = path_questionfile

    def makeQuestionCards(self, rawlines_questions):
        # make QuestionCards out of the raw lines of questions from the question file.
                
        # the question file must be organized in questions with lines
        #  1) "Q: ..:", 2) "A: ..." and 3) "C: ...".
        list_QuestionCards = []
        while rawlines_questions:
            # The categories
            line_categories = rawlines_questions.pop()
            # The answers
            # Check for additional lines -- answers could be long, written over several lines
            line_answers = ""
            rawlines_questions_temp = list(rawlines_questions)
            while not rawlines_questions_temp.pop().startswith("A:"):
                line_answers += rawlines_questions.pop()
            line_answers = rawlines_questions.pop() + "\n" + line_answers
            #DEBUG
            print("Making Question Card -- here's the line w/ answers: \n{}".format(line_answers))
            # Finally, the question
            line_question   = rawlines_questions.pop()

            if not (line_question.startswith("Q:") and line_answers.startswith("A:") and line_categories.startswith("C:")):
                raise Exception("QuestionFileFormatError: Question file at {0} doesn't have correct format 'Q: .., A: .. C: '! \n You have question line: {1} \n     answer line: {2} \n  and categories line: {3}".format(self.path_questionfile, line_question, line_answers, line_categories))

            # If multiple-choice, the answers are separeted by ";"
            # If not multiple-choice, list will hold just one item
            answers = line_answers.split(";")
            if len(answers) < 1:
                raise Exception("QuestionFileFormatError: No answers found after 'A:' in question file! You should give them ';'-separated, the first one should be the correct answer. Line in file = {}".format(line_answers))
            
            for idx_ans in range(len(answers)):
                # remove duplicated spaces
                line_ans = answers[idx_ans]
                if len(answers) > 1:
                    answers[idx_ans] = ' '.join(line_ans.split())
                # but if not-multiple choice, keep new lines
                else:
                    answers[idx_ans] = '\n'.join(' '.join(line.split()) for line in line_ans.split('\n'))
                # assure no answer is empty
                if answers[idx_ans] == "":
                    raise Exception("QuestionFileFormatError: Empty answers found among answer alternatives. Line in file = {}".format(line_answers))
                
            answers[0] = answers[0].replace("A: ", "")
            
            # The categories
            categories = [""]
            if line_categories:
                categories = line_categories.split(",")
                for idx_categ in range(len(categories)):
                    # remove duplicated spaces
                    categories[idx_categ] = ' '.join(categories[idx_categ].split()) 
                    # make sure no category is empty
                    if categories[idx_categ] == "":
                        raise Exception("QuestionFileFormatError: Empty category found among categories. They should be given comma-separated. Line in file = {}".format(line_categories))
                categories[0] = categories[0].replace("C: ", "")
            
            # Make the question card, append to list of question cards
            qCard = QuestionCard(line_question, answers, categories)
            list_QuestionCards.append(qCard)
        return list_QuestionCards

    def getListOfQuestionCards(self):
        # Open file, get lines as list
        with open(self.path_questionfile) as file_questions:
            rawlines_questions = list(filter(None, (line.rstrip() for line in file_questions)))
       
        list_QuestionCards = []
        try:
            list_QuestionCards = self.makeQuestionCards(rawlines_questions)
        except Exception as error:
            print(error)
        except:
            # default case - print message and re-raise exception
            print("Unexpected error:", sys.exc_info()[0])
            raise

        return list_QuestionCards


# Deck of QuestionCards
class Deck:

    def __init__(self):
        # the deck of cards, implemented as dictionary, {0: QuestionCard, 1: QuestionCard...}
        self.library = {}
        self.categs = []

    # Override len() and index operators, so the Deck itself can be treated like a dictionary
    def __len__(self):
        return len(self.library)
    
    def __getitem__(self, idx):
        return self.library[idx]

    def load(self, path_to_questionfile, categories = None):
        # Fill Deck with QuestionCards with the help of QuestionMaker
        # By default, fill with all questions
        # TODO: implement to fill with only certain category
        qMaker = QuestionMaker(path_to_questionfile)
        listOfQuestionCards = qMaker.getListOfQuestionCards()
        if len(self.library) == 0:
            id = 0
            for qCard in listOfQuestionCards:
                self.library[id] = qCard
                id += 1
        else:
            print("You're trying to add to a deck which isn't empty. Is that what you want to do? If yes, implement it :)")
            
        if g_DEBUG:
            print("You have now collected question cards for this deck. The cards are: ")
            self.dumpQuestions()
    
    def dumpQuestions(self):
        for key_card in self.library:
            print("\n*** Question {}: ".format(key_card+1))
            self.library[key_card].show()
            
    def dumpCategories(self):
        categs = []
        for key_card in self.library:
            categs += [ca for ca in self.library[key_card].categ if ca not in categs]
        print("")
        for ca in categs:
            print("* {}".format(ca))
    
    def filterCateg(self, categories):
        # specialise the deck by removing cards which aren't tagged with the desired category
        lib_specialised = {key:self.library[key] for key in self.library if not set(self.library[key].categ).isdisjoint(categories)}
        print("DEBUG Inside Deck::specialise >> lib_specialised = {}".format(lib_specialised))
        if len(lib_specialised) < 1:
            raise Exception("ERROR in Deck::specialise >> ended up with empty deck. None of your categories {} exist!".format(categories))
        self.library = lib_specialised
            
            

# The QuizMaster that will run the quiz
class QuizMaster:

    def __init__(self, path_to_questionfile = g_PATHQUESTIONFILE):
        self.all_questions = Deck()
        self.all_questions.load(path_to_questionfile)
        if len(self.all_questions) < 1:
            raise ValueError("ERROR in QuizMaster::__init__ >> questions loaded, but deck empty!")
            
        self.categories = [""]     # dummy choice of categories, will be set by run()
        self.current_deck = self.all_questions # set to all_questions by default, will be filtered in run() if certain categories given

        # List of items in menu. Make sure "Quit" is the last item
        self.menu = ["Take quiz", "Enter new question", "Quit"]
        
        
    def showMenu(self):
        print("\nWhat do you want to do?")

        for idx,item in enumerate(self.menu):
            print("{}: {}".format(idx+1, item))


    def run(self):

        self.showMenu()

        try:
            choice = int(input("Please enter your choice: "))
            assert 1 <= choice <= len(self.menu)
        except AssertionError:
            print("Choice not recognized (did you enter a number in the menu?) -- please try again.")
            self.run()
        except ValueError:
            print("Choice doesn't appear to be an integer. Please try again.")
            self.run()

        if choice == len(self.menu):
            print("\n*** Exiting program. Good  bye. ")
            # should maybe exit explicitly, but for now just do nothing. (Don't want "kernel has died" message...)
            #exit()
        else:
            # hard-coded choice cases for now
            if choice == 1:
                self.takeQuiz(self)
                    
            elif choice == 2:
                print("This feature is not implemented yet... Sorry.")


    def takeQuiz(self, choice):
        print("\nYou have chosen to take the quiz.")
        print("Do you want to take it only with certain categorie(s) (subject(s))? Here are the list of categories: \n")
        self.all_questions.dumpCategories()
        categ = []
        categ = str(input("Enter the categories (comma-separated). If you want to take it with all categories, just press enter:"))
        if categ:
            # remove whitespace 
            categ = ''.join(categ.split())
            # make list with categories
            categ = categ.split(",")
            print(categ)
            try: 
                self.current_deck.filterCateg(categ)
            except Exception as error:
                print(error)
                self.takeQuiz()
                    
        # Run the quiz by asking questions until user terminates it
        print("\n*** Starting quiz. Questions coming up! \n")
                       
        while self.askQuestion():
            pass
                    
        print("\n*** You have chosen to end the program. Good bye!")
        

    def askQuestion(self):
        if g_DEBUG: print("Inside QuizMaster::askQuestion()")

        # Get random card from deck, ask its question. 
        # The 'ask' method returns True or False, where False means 
        # the user wants to quit the program (not answer any more questions) 
        rand_key = random.choice(list(self.current_deck.library.keys()))
        return self.current_deck[rand_key].ask()
        
    def addQuestion(self):
        if g_DEBUG: print("Inside QuizMaster::addQuestion()")
            

### Main program sketch
if __name__ == '__main__':

    print("\nWelcome to the quizMe program!")

    #quizMaster = QuizMaster("/Users/edvinsidebo/code-projects/quizMe/Questions/testfilequestions.txt")
    quizMaster = QuizMaster(g_PATHQUESTIONFILE)
    quizMaster.run()


In [41]:
letters = 'ABCDEFG'
assert 'A' in letters
letters[:3]


foo = [1, 2]
bar = list(foo)
#bar.pop()
#print(foo)

line = "hallon \n   karl  "
print(line)
print(' '.join(line.split("\ ")))

print(foo)
bar = [2,4]
set(bar).isdisjoint(foo)

text = "hallon , ffff"
textnws = ''.join(text.split())
print(textnws)
#categs = [word for word in text.split(",")]
#categs = ''.join(categs.split())
#print(categs)
#print(' '.join(''.join(text.split(",")).split()))


hallon 
   karl  
hallon 
   karl  
[1, 2]
hallon,ffff


In [5]:
import random
dic = {0: "nisse", 1: "grej", 2: "krister"}
listofkeys = list(dic.keys())
random.shuffle(listofkeys)
print(listofkeys)
print(list(range(3)))
print(listofkeys[0:len(listofkeys)])

lolo = ["hej", "bar"]
lo = lolo.pop() 
lo += "bapappa"
print(lo)




[1, 0, 2]
[0, 1, 2]
[1, 0, 2]
barbapappa


In [47]:
'a b'.replace(" ","")

'ab'